# 빌딩 전력 소모량 예측

여러분들의 분석 스킬과 머신러닝 스킬을 활용하여 빌딩의 전력 소모량을 예측하는 모델을 만들어 주세요!

Q : 여름에 고층 빌딩을 식히는 데 비용이 얼마나 드나요?
A : 많이 듭니다! 그리고 돈 뿐만 아니라 환경 영향도 있습니다.

건물 효율성을 개선하여 비용과 배출량을 줄이기 위해 상당한 투자가 이루어지고 있습니다. 문제는 개선이 효과적이었을까요?

건물 소유주는 전력을 사용한 만큼 청구서를 발행하는데,전력 계량기가 특정 건물 유형에서는 작동하지 않아 전력 사용량 측정에도 어려움을 겪고 있습니다.재의 추정 방법은 단편화되어 있으며 확장 성이 좋지 않습니다. 일부는 특정 미터 유형을 가정하거나 다른 건물 유형에서 작동하지 않습니다.

이 대회에서는 냉수, 전기, 온수 및 증기 계량기 분야에서 건물 에너지 사용량의 정확한 모델을 개발하게됩니다. 데이터는 3 년 동안 1,000 개 이상의 건물에서 가져온 것입니다. 이러한 에너지 절약 투자에 대한 더 나은 추정을 통해 대규모 투자자와 금융 기관은 건물 효율성 향상을 위해 이 분야에 투자 할 가능성이 높아질 것입니다.


## 데이터 설명
**train.csv**

- `building_id`-건물 ID 코드
- `meter`-측정기 ID 코드
    - {0 : 전기, 1 : 냉수, 2 : 증기, 3 : 온수}
- `timestamp`-측정이 수행 된 시기
- `meter_reading`-대상 변수입니다. 
    - kWh (또는 이에 상응하는) 단위의 에너지 소비량

**test.csv**
- `row_id`-제출 파일의 행 ID
- `building_id`-건물 ID 코드
- `meter`-측정기 ID 코드
- `timestamp`-측정이 수행 된 시기

**building_meta.csv**

- `site_id`-위치 ID 코드 
    - `weather` 파일과 병합 시 사용
- `building_id`-건물 ID 코드
    - `train.csv`, `test.csv` 파일과 병합 시 사용
- `primary_use`-미국 EnergyStar 속성 유형 정의를 기반으로하는 건물의 기본 활동 범주 표시기
- `square_feet`-건물의 전체 바닥 면적
- `year_built`-개관 연도
- `floor_count`-건물의 층 수

**weather_[train/test].csv**

건물에 가장 가까운 기상 관측소의 날씨 데이터
- `site_id` - 위치 ID 코드 
- `air_temperature.csv`- 섭씨 온도
- `cloud_coverage`-구름으로 덮인 하늘 부분 ([Okta](https://en.wikipedia.org/wiki/Okta))
- `dew_ temperature`-섭씨 온도
- `precip_depth_1_hr`-밀리미터
- `sea_level_pressure`-밀리바 / 헥토 파스칼
- `wind_direction`-나침반 방향 (0-360)
- `wind_speed`-초당 미터

**submission.csv**

- `row_id`-제출 파일의 행 ID
- `meter_reading`-test.csv 파일을 예측한 결과

## 프로젝트 구성
* 데이터 로드 (load data, train.csv, test.csv)
* 데이터 전처리 및 특성 공학 (train.csv, test.csv)
* 모델 생성 및 검증 (train.csv)
* 평가 데이터 예측 (test.csv)
* 결과 제출 및 점수 확인

## 제출 이력 및 순위 조회
[평가시스템](http://manage.jaen.kr)에 접속하여 확인 가능

> 로그인 -> 프로젝트 -> 프로젝트 선택 -> 리더보드

- 접속 정보
    - ID : `~~~@sk.com`
    - PWD : `mysuni`




## 환경 설정

In [54]:
project_name = "빌딩 전력 소모량 예측" # 프로젝트 이름
class_info = "1차수 B반" # 차수 정보
email = "yong19@sk.com" # 회사 이메일을 넣어주세요. (~~~@sk.com)

## 모듈 설치 및 데이터 다운로드

In [55]:
!pip install --upgrade mySUNI

In [56]:
from mySUNI import cds

In [57]:
cds.download_project(project_name, class_info, email)


프로젝트: 빌딩 전력 소모량 예측
파일 목록

test.csv
- data/빌딩 전력 소모량 예측/test.csv

submission.csv
- data/빌딩 전력 소모량 예측/submission.csv

train.csv
- data/빌딩 전력 소모량 예측/train.csv



## 모듈 import 
필요한 모듈을 import 합니다. 필요에 따라 추가 패키지를 load 할 수 있습니다.

In [58]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Data 경로 설정
DATA_DIR = 'data'

# train 데이터셋 로드 (train.csv)
train = pd.read_csv(os.path.join(DATA_DIR, project_name, 'train.csv'))
                                 
# test 데이터셋 로드 (test.csv)
test = pd.read_csv(os.path.join(DATA_DIR, project_name, 'test.csv'))

# submission 파일 로드 (submission.csv)
submission = pd.read_csv(os.path.join(DATA_DIR, project_name, 'submission.csv'))

2개의 데이터 셋트를 나누어 제공합니다.
1. train: 학습용 데이터 세트. 정답 값이라고 할 수 있는 `label` 값이 같이 제공됩니다.
2. test: 예측용 데이터 세트. 정답 값이 빠진 `feature` 들만 제공됩니다. 우리는 이 `feature`를 활용하여 예측을 진행합니다.
3. submission: 정답 제출용 데이터 세트. 주어진 형식에 맞춰 정답 값을 기입하고 제출합니다.

In [59]:
# 학습 데이터 조회
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,597,0,2016-01-01 12:00:00,71.000,4,9,189425,1994.0,4.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
1,620,0,2016-01-01 12:00:00,1122.880,4,0,220703,2011.0,9.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
2,631,0,2016-01-01 12:00:00,8.685,4,0,23533,1906.0,3.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
3,641,0,2016-01-01 12:00:00,393.528,4,0,109588,1994.0,8.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
4,575,0,2016-01-01 18:00:00,25.625,4,13,33167,1963.0,4.0,7.2,2.0,-7.2,0.0,1022.5,120.0,2.6


In [60]:
# 평가 데이터 조회
test.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,575,0,2016-01-01 12:00:00,4,5,33167,1963.0,4.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
1,585,0,2016-01-01 12:00:00,4,4,132394,1949.0,7.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
2,609,0,2016-01-01 12:00:00,4,0,141461,1907.0,6.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
3,652,0,2016-01-01 12:00:00,4,0,150887,1959.0,9.0,5.0,0.0,-9.4,0.0,1020.6,0.0,0.0
4,597,0,2016-01-01 18:00:00,4,4,189425,1994.0,4.0,7.2,2.0,-7.2,0.0,1022.5,120.0,2.6


In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22514 entries, 0 to 22513
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   building_id         22514 non-null  int64  
 1   meter               22514 non-null  int64  
 2   timestamp           22514 non-null  object 
 3   meter_reading       22514 non-null  float64
 4   site_id             22514 non-null  int64  
 5   primary_use         22514 non-null  int64  
 6   square_feet         22514 non-null  int64  
 7   year_built          22514 non-null  float64
 8   floor_count         22514 non-null  float64
 9   air_temperature     22514 non-null  float64
 10  cloud_coverage      22514 non-null  float64
 11  dew_temperature     22514 non-null  float64
 12  precip_depth_1_hr   22514 non-null  float64
 13  sea_level_pressure  22514 non-null  float64
 14  wind_direction      22514 non-null  float64
 15  wind_speed          22514 non-null  float64
dtypes: f

In [62]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9666 entries, 0 to 9665
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   building_id         9666 non-null   int64  
 1   meter               9666 non-null   int64  
 2   timestamp           9666 non-null   object 
 3   site_id             9666 non-null   int64  
 4   primary_use         9666 non-null   int64  
 5   square_feet         9666 non-null   int64  
 6   year_built          9666 non-null   float64
 7   floor_count         9666 non-null   float64
 8   air_temperature     9666 non-null   float64
 9   cloud_coverage      9666 non-null   float64
 10  dew_temperature     9666 non-null   float64
 11  precip_depth_1_hr   9666 non-null   float64
 12  sea_level_pressure  9666 non-null   float64
 13  wind_direction      9666 non-null   float64
 14  wind_speed          9666 non-null   float64
dtypes: float64(9), int64(5), object(1)
memory usage: 1.1+ M

In [63]:
features = [
     'building_id',
     'primary_use',
     'square_feet',
     'year_built',
     'floor_count',
     'air_temperature',
     'cloud_coverage',
     'dew_temperature',
     'precip_depth_1_hr',
     'sea_level_pressure',
     'wind_direction',
     'wind_speed'
] # 숫자형 컬럼이름만 지정 (문자열 컬럼 제거, 분산 0 컬럼 제거)
X = train[features]
Y = train['meter_reading']

In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)


In [65]:


#두개의 값이 너무나 비슷하다. 과대적합으로 예상된다.

In [66]:
from sklearn.ensemble import RandomForestRegressor
Model = RandomForestRegressor().fit(X,Y)

model.score(x_train, y_train), model.score(x_test, y_test) # 학습 및 평가 데이터로 성능평가 진행 > R^2

(0.9966701134475764, 0.9970281660346396)

In [68]:
y_pre = model.predict(x_test)
y_pre

array([  0.33765 ,  77.842376, 114.341917, ...,  34.148159,  60.5775  ,
        31.68995 ])

In [69]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pre)
mae

8.345614079054897

In [70]:
pred = model.predict(test[features])
pred

array([ 27.875   , 227.88045 , 498.31313 , ..., 214.26628 ,  63.479447,
       313.26    ])

In [73]:
pred

array([ 27.875   , 227.88045 , 498.31313 , ..., 214.26628 ,  63.479447,
       313.26    ])

In [74]:
submission['meter_reading'] = pred # test.csv를 예측한 결과 대입
submission

,building_id,meter_reading
0,575,27.875000
1,585,227.880450
2,609,498.313130
3,652,106.841506
4,597,148.772950
...,...,...
9661,619,186.637630
9662,651,46.989700
9663,619,214.266280
9664,630,63.479447


# 순서대로 평가가 진행됩니다. 실행은 1회만 해주세요.

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)

In [75]:
cds.submit(submission) # 제출 파일 생성 및 결과 제출

파일을 저장하였습니다. 파일명: submission-21-32-41.csv
제출 여부 :success
오늘 제출 횟수 : 1
제출 결과:0.31257048356098555


# 축하드립니다. 모델의 성능을 개선하여 더 좋은 점수를 획득해보세요!